# Omniscient Mozart

This is a colab for demonstrating the python package `omnizart` developed by [MCTLab](https://sites.google.com/view/mctl/home).

Github repository can be found in [Music-and-Culture-Technology-Lab/omnizart](https://github.com/Music-and-Culture-Technology-Lab/omnizart).

Official documentation page can be found in [omnizart-doc](https://music-and-culture-technology-lab.github.io/omnizart-doc/)

## Environment Setup

**IMPORTANT!** Don't restart the session when the pop up window ask you to! Just press cancel and continue.

In [ ]:
#@title Environment Setup

!sudo apt-get install python3.8 python3.8-distutils python3.8-dev
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8 1
!sudo apt-get install python3-pip
!pip install numpy Cython
!sudo apt-get install libsndfile-dev fluidsynth ffmpeg

!pip install git+https://github.com/Music-and-Culture-Technology-Lab/omnizart.git
!omnizart download-checkpoints
!pip install yt-dlp numpy==1.23.5

import sys
sys.path.insert(0, '/usr/local/lib/python3.8/dist-packages')

# Choose an Audio

Either upload your own MP3 file, or choose from YouTube.


In [ ]:
#@title Upload MP3 File
import os
from google.colab import files
from IPython import display as dsp

ff = files.upload()
uploaded_audio = list(ff.keys())[0].replace(".mp3", "")

!test -f "$uploaded_audio".wav && rm "$uploaded_audio".wav
!ffmpeg -i "$uploaded_audio".mp3 "$uploaded_audio".wav &>/dev/null

dsp.Audio(uploaded_audio + ".mp3") if os.path.exists(uploaded_audio + ".mp3") else None

In [ ]:
#@title Choose from YouTube
import os
from google.colab import files
from IPython import display as dsp

url = input("Enter your YouTube link: ")

try:
  id = url.split("watch?v=")[1].split("&")[0]
  vid = dsp.YouTubeVideo(id)
  dsp.display(vid)
except Exception:
  pass

print("Downloading...")

!yt-dlp -x --audio-format mp3 --no-playlist "$url"
!yt-dlp --get-filename --no-playlist "$url" > tmp

uploaded_audio = os.path.splitext(open("tmp").readline().strip())[0]
!ffmpeg -i "$uploaded_audio".mp3 "$uploaded_audio".wav &> /dev/null

print(f"Finished: {uploaded_audio}")

# Transcribe the Audio

There are several modes you can choose.
* `music-piano`: transcribe piano solo clips.
* `music-assemble`: transcribe classical assemble pieces.
* `chord`: transcribe chord progressions.
* `drum`: transcribe drum percussion in the audio.
* `vocal`: transcribe note-level vocal notes.
* `vocal-contour`: transcribe frame-level vocal pitch contour.
* `beat`: transcribe beat and down beat positions on symbolic domain *(see note 1)*.

## Notes
1. The beat module only supports MIDI inputs, and thus you have to upload the MIDI file through the **Upload MP3 File** block. 

In [ ]:
#@title Transcribe

mode = "vocal" #@param ["music-piano", "music-piano-v2", "music-assemble", "chord", "drum", "vocal", "vocal-contour", "beat"]

model = ""
if mode.startswith("music"):
  mode_list = mode.split("-")
  mode = mode_list[0]
  model = "-".join(mode_list[1:])

model_path = {
    "piano": "Piano",
    "piano-v2": "PianoV2",
    "assemble": "Stream",
    "pop-song": "Pop",
    "": None
}[model]

if model_path:
  !omnizart "$mode" transcribe "$uploaded_audio".wav --model-path "$model_path"
else:
  !omnizart "$mode" transcribe "$uploaded_audio".wav

# Synthesize MIDI and play
import wave
import numpy as np
from pretty_midi import PrettyMIDI
from omnizart.remote import download_large_file_from_google_drive


def write_audio_file(filename, sample_rate, data):
    with wave.open(filename, 'w') as wf:
        # Set the parameters for the wave file
        wf.setnchannels(1)  # Mono
        wf.setsampwidth(2)  # 2 bytes per sample (16-bit)
        wf.setframerate(sample_rate)

        # Convert the data type if necessary
        if data.dtype != np.int16:
            data = np.int16(data * 32767)  # Scale to 16-bit integer range

        # Write the audio frames to the file
        wf.writeframes(data.tobytes())


SF2_FILE = "general_soundfont.sf2"
if not os.path.exists(SF2_FILE):
  print("Downloading soundfont...")
  !curl "https://ftp.osuosl.org/pub/musescore/soundfont/MuseScore_General/MuseScore_General.sf2" -o $SF2_FILE

synth_name = f"{uploaded_audio}_synth.wav"
if mode == "vocal-contour":
  os.rename(f"{uploaded_audio}_trans.wav", f"{uploaded_audio}_synth.wav")
else:
  print("Synthesizing MIDI...")
  midi = PrettyMIDI(f"{uploaded_audio}.mid")
  raw_wav = midi.fluidsynth(fs=44100, sf2_path=SF2_FILE)
  write_audio_file(synth_name, 44100, raw_wav)

!ffmpeg -i "$synth_name" "tmp_synth.mp3" &> /dev/null
!mv tmp_synth.mp3 "$uploaded_audio"_synth.mp3

out_name = synth_name.replace(".wav", ".mp3")
print(f"Finished: {out_name}")
dsp.Audio(out_name)

# Download the Transribed MIDI/MP3

In [ ]:
#@title Download MIDI

files.download(f"{uploaded_audio}.mid")

In [ ]:
#@title Download MP3

files.download(out_name)